In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import math
import random

import numpy as np
import torch
import socialforce

[Sven Kreiss](https://www.svenkreiss.com), March 5, 2020

# Fit PedPed Potential

In [ ]:
# the "pedestrian of interest" is beta and the probe pedestrians are alpha

x1 = np.linspace(-1.0, 2.0, 101, dtype=np.float32)
x2 = np.linspace(-1.0, 1.0, 101, dtype=np.float32)
xx1, xx2 = np.meshgrid(x1, x2)
r_ab_probe = torch.from_numpy(np.stack((xx1, xx2), axis=-1)).view(-1, 2).unsqueeze(1)

speeds_b = torch.ones((r_ab_probe.shape[1],)) * 1.0
desired_directions_b = torch.zeros((r_ab_probe.shape[1], 2))
desired_directions_b[:, 0] = 1.0

def contour(ax, V):
    ax.axhline(0.0, ls='dotted', color='gray')
    ax.axvline(0.0, ls='dotted', color='gray')

    with torch.no_grad():
        values = V.value_r_ab(r_ab_probe, speeds_b, desired_directions_b, 1.0)
        values -= torch.min(values)
    values = values.view((len(x1), len(x2)))
    ax.clabel(
        ax.contour(x1, x2, values, levels=np.linspace(0.1, 5.0, 50), vmax=1.0), 
        inline=1, fontsize=10)

    ax.plot([0.0], [0.0], 'o', label='pedestrian', markersize=5.0, color='black')
    ax.set_xlabel('$x_1$ [m]')
    ax.set_ylabel('$x_2$ [m]')
    ax.legend()

with socialforce.show.canvas('images/twod_pedped.png') as ax:
    contour(ax, socialforce.potentials.PedPedPotential2D())

In [ ]:
with socialforce.show.canvas() as ax:
    contour(ax, socialforce.potentials.PedPedPotential2D(field_of_view=socialforce.FieldOfView()))

## Fitting to 1, 5 and 20 Circle Scenarios each with Two Pedestrians

The preferred speed needs to be varied. Otherwise the symmetry of the problem creates unrealistic scenarios where the two pedestrians get stuck.

In [ ]:
def generate_gt(n):
    torch.manual_seed(42)
    np.random.seed(42)

    # ped0 always left to right
    ped0 = np.array([-5.0, 0.0, 1.0, 0.0, 5.0, 0.0])

    generator_initial_states = []
    for theta in np.random.rand(n) * 2.0 * math.pi:
        # ped1 at a random angle with +/-20% speed variation
        c, s = np.cos(theta), np.sin(theta)
        r = np.array([[c, -s], [s, c]])
        ped1 = np.concatenate((
            np.matmul(r, ped0[0:2]),
            np.matmul(r, ped0[2:4]) * (0.8 + np.random.rand(1) * 0.4),
            np.matmul(r, ped0[4:6]),
        ))
        generator_initial_states.append(
            np.stack((ped0, ped1))
        )

    generator_ped_ped = socialforce.PedPedPotential(2.1)
    with torch.no_grad():
        trajectories = [
            socialforce.Simulator(initial_state, ped_ped=generator_ped_ped).run(21)
            for initial_state in generator_initial_states
        ]
    return generator_ped_ped, trajectories

generator_pedped_1, scenarios_1 = generate_gt(1)
generator_pedped_5, scenarios_5 = generate_gt(5)
generator_pedped_20, scenarios_20 = generate_gt(20)

In [ ]:
with socialforce.show.track_canvas(ncols=3, figsize=(12, 4)) as (ax1, ax2, ax3):
    socialforce.show.states(ax1, scenarios_1[0])

    socialforce.show.states(ax2, scenarios_5[-1])
    for scene in scenarios_5[:-1]:
        socialforce.show.states(ax2, scene, alpha=0.1)

    socialforce.show.states(ax3, scenarios_20[-1])
    for scene in scenarios_20[:-1]:
        socialforce.show.states(ax3, scene, alpha=0.1)

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios_1)
V = socialforce.PedPedPotentialMLP().double()
initial_parameters = V.state_dict()

def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=10.0)
socialforce.Trainer(simulator_factory, opt, true_experience, batch_size=1).loop(50)

In [ ]:
# make plots of result
visualize('images/mlp_circle_n{}_'.format(1), V, initial_parameters, V.state_dict(), V_gen=generator_pedped_1)

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios_5)
V = socialforce.PedPedPotentialMLP().double()
initial_parameters = V.state_dict()

def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=1.0)
socialforce.Trainer(simulator_factory, opt, true_experience, batch_size=1).loop(100)

In [ ]:
visualize('images/mlp_circle_n{}_'.format(5), V, initial_parameters, V.state_dict(), V_gen=generator_pedped_5)